In [33]:
import pandas as pd
import numpy as np
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpBinary, LpStatus

def optimize_warehouse_storage(data_path):
    pd.options.display.max_rows = None  # Set to None to display all rows
    pd.options.display.max_columns = None  # Set to display all columns
    pd.set_option('display.expand_frame_repr', False)  # Prevent DataFrame wrapping

    # Load and preprocess data
    df = pd.read_csv(data_path)
    
    # Extract key parameters
    num_skus = len(df['s'].unique())
    num_locations = len(df['l'].unique())
    max_bins = int(df['ni'].dropna().max())
    
    # Create mapping dictionaries for locations and SKUs
    location_map = {loc: idx for idx, loc in enumerate(df['l'].unique())}
    sku_map = {sku: idx for idx, sku in enumerate(df['s'].unique())}
    
    # Initialize optimization problem
    prob = LpProblem("Warehouse_Optimization", LpMinimize)
    
    # Decision variables
    y = LpVariable.dicts("storage_assignment",
                         ((s, i, t) for s in range(num_skus)
                          for i in range(num_locations)
                          for t in range(int(max_bins))),
                         cat=LpBinary)
    
    z_pick = LpVariable.dicts("picking_distance", range(len(df)), lowBound=0)
    z_replenish = LpVariable.dicts("replenish_distance", range(len(df)), lowBound=0)
    
    # Objective function: Minimize total distance (picking + replenishment)
    avg_walking_speed = 1.0  # meters per second
    prob += (lpSum(z_pick[k] + z_replenish[k] for k in range(len(df))) * 
            (1/avg_walking_speed))  # Convert to time
    
    # Constraints
    for s in range(num_skus):
        prob += lpSum(y[s,i,t] for i in range(num_locations) 
                      for t in range(int(max_bins))) == 1

    for i in range(num_locations):
        prob += lpSum(y[s,i,t] * t for s in range(num_skus)
                      for t in range(int(max_bins))) <= df['ni'].iloc[i]

    for k in range(len(df)):
        sku_idx = sku_map[df['s'].iloc[k]]
        prob += z_pick[k] >= lpSum(2 * df['hi'].iloc[i] * y[sku_idx,i,t]
                                   for i in range(num_locations)
                                   for t in range(int(max_bins)))

        prob += z_replenish[k] >= lpSum(
            2 * df['r'].iloc[i] * np.ceil(df['ds'].iloc[k] / (df['qs'].iloc[k] * t)) * y[sku_idx,i,t]
            for i in range(num_locations)
            for t in range(int(max_bins)) if df['qs'].iloc[k] * t > 0)

    # Solve the problem
    prob.solve()

    # Process results
    results = []
    if LpStatus[prob.status] == 'Optimal':
        for s in range(num_skus):
            for i in range(num_locations):
                for t in range(int(max_bins)):
                    if y[s,i,t].varValue > 0.5:
                        results.append({
                            'SKU': list(sku_map.keys())[s],
                            'Location': list(location_map.keys())[i],
                            'Bin': t + 1,
                            'Distance_Pick': z_pick[s].varValue,
                            'Distance_Replenish': z_replenish[s].varValue,
                            'Time_Pick': z_pick[s].varValue / avg_walking_speed,
                            'Time_Replenish': z_replenish[s].varValue / avg_walking_speed
                        })
        
        results_df = pd.DataFrame(results)
        print(results_df)
        total_distance = sum(z_pick[k].varValue + z_replenish[k].varValue for k in range(len(df)))
        total_time = total_distance / avg_walking_speed
        print(f"\nTotal Distance: {total_distance:.2f} meters")
        print(f"Total Time: {total_time:.2f} seconds")
    else:
        print("No optimal solution found")

# Usage example
if __name__ == "__main__":
    optimize_warehouse_storage('Book12.csv')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/f8/sfvcnkwx62g3v6xnb7x8g7580000gn/T/b6c340ce8aa546ac8955f06c5697c92e-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/f8/sfvcnkwx62g3v6xnb7x8g7580000gn/T/b6c340ce8aa546ac8955f06c5697c92e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 254 COLUMNS
At line 31055 RHS
At line 31305 BOUNDS
At line 36850 ENDATA
Problem MODEL has 249 rows, 5698 columns and 19558 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 677.6 - 0.05 seconds
Cgl0004I processed model has 249 rows, 3542 columns (3388 integer (3388 of which binary)) and 13398 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 677.6
Cbc0038I Relaxing continuous gives 677.6
Cbc0038I Before mini b

In [11]:
import pandas as pd
import numpy as np
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpBinary, LpStatus

def optimize_warehouse_storage(data_path):
    pd.options.display.max_rows = None  # Display all rows
    pd.options.display.max_columns = None  # Display all columns
    pd.set_option('display.expand_frame_repr', False)  # Prevent DataFrame wrapping

    # Load and preprocess data
    df = pd.read_csv(data_path)
    
    # Assuming 'order_frequency' needs to be calculated from another data source or column
    # Example: df['order_frequency'] = calculate_order_frequency(df)
    # For now, simulate this frequency as a random integer (placeholder)
    df['order_frequency'] = np.random.randint(1, 100, size=len(df))

    # Sort SKUs by their frequency of appearance in descending order
    df.sort_values('order_frequency', ascending=False, inplace=True)

    # Extract key parameters
    skus = df['s'].unique()
    locations = df['l'].unique()
    max_bins = {location: df[df['l'] == location]['ni'].max() for location in locations}

    # Initialize the MILP problem
    prob = LpProblem("Warehouse_Optimization", LpMinimize)
    
    # Decision variables
    # y[s, i, t] is 1 if SKU s is assigned to bin t at location i
    y = LpVariable.dicts("bin_assignment",
                         ((s, i, t) for s in skus for i in locations for t in range(1, max_bins[i] + 1)),
                         cat='Binary')

    # Objective function: Minimize the distance traveled for picking and replenishment
    # Distance factors can be defined similarly to df['hi'] and df['r'], adjusted by t
    prob += lpSum(df.loc[df['s'] == s, 'hi'].values[0] * y[s, i, t] + 
                  df.loc[df['s'] == s, 'r'].values[0] * y[s, i, t] * t 
                  for s in skus for i in locations for t in range(1, max_bins[i] + 1))

    # Constraints
    # Each SKU s must be assigned to exactly one location and one bin
    for s in skus:
        prob += lpSum(y[s, i, t] for i in locations for t in range(1, max_bins[i] + 1)) == 1

    # Ensure the bin capacities are not exceeded at each location
    for i in locations:
        for t in range(1, max_bins[i] + 1):
            prob += lpSum(y[s, i, t] for s in skus) <= 1  # Assuming each bin can store one SKU

    # Solve the problem
    prob.solve()

    # Output the results
    results = []
    for s in skus:
        for i in locations:
            for t in range(1, max_bins[i] + 1):
                if y[s, i, t].value() == 1:
                    results.append({'SKU': s, 'Location': i, 'Bin': t})

    results_df = pd.DataFrame(results)
    print(results_df)

# Usage example
if __name__ == "__main__":
    optimize_warehouse_storage('Book12.csv')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/f8/sfvcnkwx62g3v6xnb7x8g7580000gn/T/b5d79c704ca04947a437bd23175f1a84-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/f8/sfvcnkwx62g3v6xnb7x8g7580000gn/T/b5d79c704ca04947a437bd23175f1a84-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 130 COLUMNS
At line 18611 RHS
At line 18737 BOUNDS
At line 22434 ENDATA
Problem MODEL has 125 rows, 3696 columns and 7392 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.01 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.02   (Wallclock seconds):       0.04

                                SKU       Location  Bin
0                         Stamp Pad  Rak D Level 4    1
1                              Paku  R

In [36]:
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpBinary

def optimize_warehouse_storage(data_path, travel_time_per_unit_distance):
    pd.options.display.max_rows = None  # Display all rows
    pd.options.display.max_columns = None  # Display all columns
    pd.set_option('display.expand_frame_repr', False)  # Prevent DataFrame wrapping

    # Load and preprocess data
    df = pd.read_csv(data_path)
    
    df.sort_values('ds', ascending=False, inplace=True)

    # Extract key parameters
    skus = df['s'].unique()
    locations = df['l'].unique()
    max_bins = {location: df[df['l'] == location]['ni'].max() for location in locations}

    # Initialize the MILP problem
    prob = LpProblem("Warehouse_Optimization", LpMinimize)
    
    # Decision variables
    y = LpVariable.dicts("bin_assignment",
                         ((s, i, t) for s in skus for i in locations for t in range(1, max_bins[i] + 1)),
                         cat='Binary')

    # Objective function: Minimize the total travel time using the 'zk' column for distance
    prob += lpSum(df.loc[df['s'] == s, 'zk (m)'].values[0] * travel_time_per_unit_distance * y[s, i, t]
                  for s in skus for i in locations for t in range(1, max_bins[i] + 1))

    # Constraints: Each SKU s must be assigned to exactly one location and one bin
    for s in skus:
        prob += lpSum(y[s, i, t] for i in locations for t in range(1, max_bins[i] + 1)) == 1

    # Ensure the bin capacities are not exceeded at each location
    for i in locations:
        for t in range(1, max_bins[i] + 1):
            prob += lpSum(y[s, i, t] * df.loc[df['s'] == s, 'qs'].values[0] for s in skus) <= df.loc[df['l'] == i, 'ni'].max()

    # Solve the problem
    prob.solve()

    # Output the results
    total_distance = 0
    total_time = 0
    results = []
    for s in skus:
        for i in locations:
            for t in range(1, max_bins[i] + 1):
                if y[s, i, t].value() == 1:
                    distance = df.loc[df['s'] == s, 'zk (m)'].values[0]
                    time = distance * travel_time_per_unit_distance
                    total_distance += distance
                    total_time += time
                    results.append({'SKU': s, 'Location': i, 'Bin': t, 'Distance': distance, 'Time': time})

    results_df = pd.DataFrame(results)
    print(results_df)
    print(f"Total Distance: {total_distance} meters")
    print(f"Total Time: {total_time} time units")

# Usage example
if __name__ == "__main__":
    optimize_warehouse_storage('Book12.csv', 0.05)  # travel_time_per_unit_distance is a hypothetical value

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/f8/sfvcnkwx62g3v6xnb7x8g7580000gn/T/9faef47e9a4d4c4bbc36eb420a8e1f9c-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/f8/sfvcnkwx62g3v6xnb7x8g7580000gn/T/9faef47e9a4d4c4bbc36eb420a8e1f9c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 130 COLUMNS
At line 18611 RHS
At line 18737 BOUNDS
At line 22434 ENDATA
Problem MODEL has 125 rows, 3696 columns and 7392 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.04

                     SKU       Location  Bin  Distance  Time
0  Penghapus Papan Tulis  Rak C Level 4    2       4.4  0.22
1         Spidol Snowman  Ra

In [24]:
data = pd.read_csv('Book12.csv')

In [30]:
data.head()

,s,l,hi,r,o,qk,qs,ds,ni,i (m),zk (m),i (detik),zk (detik)
0,Spidol Snowman,Rak A Level 3,4.8,4.8,ATK,105,492,105,3,4.8,4.8,130,160
1,Puplen Snowman,Rak A Level 3,4.8,4.8,ATK,99,720,99,3,4.8,4.8,130,160
2,Pulpen faster,Rak A Level 3,4.8,4.8,ATK,105,36,105,3,4.8,4.8,130,160
3,Pulpen Standard,Rak A Level 3,4.8,4.8,ATK,94,36,94,3,4.8,4.8,130,160
4,Spidol Warna,Rak A Level 3,4.8,4.8,ATK,112,10,112,3,4.8,4.8,130,160


In [39]:
import pandas as pd
import numpy as np
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpBinary

def optimize_warehouse_storage(data_path, travel_time_per_unit_distance):
    pd.options.display.max_rows = None  # Display all rows
    pd.options.display.max_columns = None  # Display all columns
    pd.set_option('display.expand_frame_repr', False)  # Prevent DataFrame wrapping

    # Load and preprocess data
    df = pd.read_csv(data_path)

    # Extract key parameters
    skus = df['s'].unique()
    locations = df['l'].unique()
    max_bins = {location: df[df['l'] == location]['ni'].max() for location in locations}

    # Initialize the MILP problem
    prob = LpProblem("Warehouse_Optimization", LpMinimize)
    
    # Decision variables
    y = LpVariable.dicts("bin_assignment",
                         ((s, i, t) for s in skus for i in locations for t in range(1, max_bins[i] + 1)),
                         cat='Binary')

    # Objective function: Minimize the total distance and time preparing orders
    prob += lpSum(df.loc[df['s'] == s, 'zk (m)'].values[0] * y[s, i, t] +
                  df.loc[df['s'] == s, 'zk (detik)'].values[0] * y[s, i, t]
                  for s in skus for i in locations for t in range(1, max_bins[i] + 1))

    # Constraints: Each SKU s must be assigned to exactly one bin in one location
    for s in skus:
        prob += lpSum(y[s, i, t] for i in locations for t in range(1, max_bins[i] + 1)) == 1

    # Ensure the bin capacities are not exceeded at each location
    for i in locations:
        for t in range(1, max_bins[i] + 1):
            prob += lpSum(y[s, i, t] for s in skus) <= 1  # Assuming each bin can store one SKU

    # Solve the problem
    prob.solve()

    # Output the results
    total_distance = 0
    total_time = 0
    results = []
    for s in skus:
        for i in locations:
            for t in range(1, max_bins[i] + 1):
                if y[s, i, t].value() == 1:
                    distance = df.loc[df['s'] == s, 'zk (m)'].values[0]
                    time = df.loc[df['s'] == s, 'zk (detik)'].values[0]
                    total_distance += distance
                    total_time += time
                    results.append({'SKU': s, 'Location': i, 'Bin': t, 'Distance': distance, 'Time': time})

    results_df = pd.DataFrame(results)
    print(results_df)
    print(f"Total Distance: {total_distance} meters")
    print(f"Total Time: {total_time} seconds")

# Usage example
if __name__ == "__main__":
    optimize_warehouse_storage('Book12.csv', 0.05)  # travel_time_per_unit_distance is a hypothetical value


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/f8/sfvcnkwx62g3v6xnb7x8g7580000gn/T/b6a29d0eaacf4112a7bc33d933704089-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/f8/sfvcnkwx62g3v6xnb7x8g7580000gn/T/b6a29d0eaacf4112a7bc33d933704089-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 130 COLUMNS
At line 18611 RHS
At line 18737 BOUNDS
At line 22434 ENDATA
Problem MODEL has 125 rows, 3696 columns and 7392 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.02   (Wallclock seconds):       0.03

                                SKU       Location  Bin  Distance  Time
0                    Spidol Snowman  Rak C Level 2    1       4.8   160
1     

In [44]:
import pandas as pd
import numpy as np
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpBinary

def optimize_warehouse_storage(data_path, travel_time_per_unit_distance):
    pd.options.display.max_rows = None  # Display all rows
    pd.options.display.max_columns = None  # Display all columns
    pd.set_option('display.expand_frame_repr', False)  # Prevent DataFrame wrapping

    # Load and preprocess data
    df = pd.read_csv(data_path)

    # Extract key parameters
    skus = df['s'].unique()
    locations = df['l'].unique()
    max_bins = {location: df[df['l'] == location]['ni'].max() for location in locations}

    # Initialize the MILP problem
    prob = LpProblem("Warehouse_Optimization", LpMinimize)
    
    # Decision variables
    y = LpVariable.dicts("bin_assignment",
                         ((s, i, t) for s in skus for i in locations for t in range(1, max_bins[i] + 1)),
                         cat='Binary')

    # Objective function: Minimize the total travel time and distance
    prob += lpSum(df.loc[df['s'] == s, 'zk (m)'].values[0] * travel_time_per_unit_distance * y[s, i, t] +
                  df.loc[df['s'] == s, 'i (m)'].values[0] * travel_time_per_unit_distance * y[s, i, t]
                  for s in skus for i in locations for t in range(1, max_bins[i] + 1))

    # Constraints: Each SKU s must be assigned to exactly one location and one bin
    for s in skus:
        prob += lpSum(y[s, i, t] for i in locations for t in range(1, max_bins[i] + 1)) == 1

    # Ensure the bin capacities are not exceeded at each location
    for i in locations:
        for t in range(1, max_bins[i] + 1):
            prob += lpSum(y[s, i, t] for s in skus) <= 1  # Assuming each bin can store one SKU

    # Solve the problem
    prob.solve()

    # Output the results
    total_distance = 0
    total_time = 0
    results = []
    for s in skus:
        for i in locations:
            for t in range(1, max_bins[i] + 1):
                if y[s, i, t].value() == 1:
                    distance_zk = df.loc[df['s'] == s, 'zk (m)'].values[0]
                    time_zk = df.loc[df['s'] == s, 'zk (detik)'].values[0]
                    distance_i = df.loc[df['s'] == s, 'i (m)'].values[0]
                    time_i = df.loc[df['s'] == s, 'i (detik)'].values[0]
                    total_distance += distance_zk + distance_i
                    total_time += time_zk + time_i
                    results.append({'SKU': s, 'Location': i, 'Bin': t, 'Distance zk': distance_zk, 'Time zk': time_zk, 'Distance i': distance_i, 'Time i': time_i})

    results_df = pd.DataFrame(results)
    print(results_df)
    print(f"Total Distance: {total_distance} meters")
    print(f"Total Time: {total_time} seconds")

# Usage example
if __name__ == "__main__":
    optimize_warehouse_storage('Book12.csv', 1)  # travel_time_per_unit_distance is a hypothetical value

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/f8/sfvcnkwx62g3v6xnb7x8g7580000gn/T/bae0d42c1dcf4cd99c525b33b671cb6d-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/f8/sfvcnkwx62g3v6xnb7x8g7580000gn/T/bae0d42c1dcf4cd99c525b33b671cb6d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 130 COLUMNS
At line 18611 RHS
At line 18737 BOUNDS
At line 22434 ENDATA
Problem MODEL has 125 rows, 3696 columns and 7392 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.02   (Wallclock seconds):       0.02

                                SKU       Location  Bin  Distance zk  Time zk  Distance i  Time i
0                    Spidol Snowman  Rak C Level 3  